In [2]:
import requests, json, logging, re
import pandas as pd

from bs4 import BeautifulSoup
from io import StringIO, BytesIO
/Users/bryanadams/jadeuc_git/scratch_code/redfin_api_calls/redfin_test.ipynb
import sys
sys.path.append('..')
import pyredfin

logging.basicConfig(level=logging.DEBUG)

In [3]:
client=pyredfin.Redfin()

In [ ]:
response=client.search('22030')

In [5]:
region_id = client.get_region_id(response)

In [6]:
region_id

'8729'

In [7]:
table_content=client.get_search_results_table(region_id=region_id, region_type=2, sold_within_days=0)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): redfin.com:443
DEBUG:urllib3.connectionpool:https://redfin.com:443 "GET /stingray/api/gis-csv?al=1&has_deal=False&has_dishwasher=False&has_laundry_facility=False&has_laundry_hookups=False&has_parking=False&has_pool=False&has_short_term_lease=False&include_pending_homes=False&isRentals=False&is_furnished=False&market=dc&num_homes=5000&ord=redfin-recommended-asc&page_number=1&region_id=8729&region_type=2&sold_within_days=0&status=9&travel_with_traffic=False&travel_within_region=False&uipt=1%2C2%2C3%2C4%2C5%2C6%2C7%2C8&utilities_included=False&v=8 HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.redfin.com:443
DEBUG:urllib3.connectionpool:https://www.redfin.com:443 "GET /stingray/api/gis-csv?al=1&has_deal=False&has_dishwasher=False&has_laundry_facility=False&has_laundry_hookups=False&has_parking=False&has_pool=False&has_short_term_lease=False&include_pending_homes=False&isRentals=False&is_fu

In [8]:
df=pd.read_csv(BytesIO(table_content))

# for sale

# has sf=1,2,3,5,6,7 and missing sold_within_days=0 so I will need to add some more logic and pop off keys and add keys if search for sale

https://www.redfin.com/stingray/api/gis-csv?al=1&has_deal=false&has_dishwasher=false&has_laundry_facility=false&has_laundry_hookups=false&has_parking=false&has_pool=false&has_short_term_lease=false&include_pending_homes=false&isRentals=false&is_furnished=false&market=dc&num_homes=350&ord=redfin-recommended-asc&page_number=1&region_id=21812&region_type=6&sf=1,2,3,5,6,7&status=9&travel_with_traffic=false&travel_within_region=false&uipt=1,2,3,4,5,6,7,8&utilities_included=false&v=8

# sold

al=1&has_deal=False&has_dishwasher=False&has_laundry_facility=False&has_laundry_hookups=False&has_parking=False&has_pool=False&has_short_term_lease=False&include_pending_homes=False&isRentals=False&is_furnished=False&market=dc&num_homes=350&ord=redfin-recommended-asc&page_number=1&region_id=21812&region_type=2&sold_within_days=0&status=9&travel_with_traffic=False&travel_within_region=False&uipt=1%2C2%2C3%2C4%2C5%2C6%2C7%2C8&utilities_included=False&v=8

In [ ]:
df.head()

In [10]:
property_id=df['URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)'][0].rsplit('/',1)[1]

In [11]:
response=client.get_property_information(property_id=property_id)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): redfin.com:443
DEBUG:urllib3.connectionpool:https://redfin.com:443 "GET /stingray/api/home/details/belowTheFold?accessLevel=1&propertyId=9741653&listingId= HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.redfin.com:443
DEBUG:urllib3.connectionpool:https://www.redfin.com:443 "GET /stingray/api/home/details/belowTheFold?accessLevel=1&propertyId=9741653&listingId= HTTP/1.1" 200 None


In [12]:
response['payload']['amenitiesInfo']['addressInfo'].keys()

dict_keys(['isFMLS', 'street', 'city', 'state', 'zip', 'countryCode'])

In [13]:
response=client.get_property_images(property_id=property_id)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): redfin.com:443
DEBUG:urllib3.connectionpool:https://redfin.com:443 "GET /stingray/api/home/details/aboveTheFold?accessLevel=1&propertyId=9741653&listingId= HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.redfin.com:443
DEBUG:urllib3.connectionpool:https://www.redfin.com:443 "GET /stingray/api/home/details/aboveTheFold?accessLevel=1&propertyId=9741653&listingId= HTTP/1.1" 200 None


In [14]:
client.get_property_timeseries_estimate(property_id=property_id)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): redfin.com:443
DEBUG:urllib3.connectionpool:https://redfin.com:443 "GET /stingray/api/home/details/avmHistoricalData?accessLevel=1&propertyId=9741653&listingId= HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.redfin.com:443
DEBUG:urllib3.connectionpool:https://www.redfin.com:443 "GET /stingray/api/home/details/avmHistoricalData?accessLevel=1&propertyId=9741653&listingId= HTTP/1.1" 200 374


{'Sep 18': 621350,
 'Oct 18': 615774,
 'Nov 18': 611240,
 'Dec 18': 613386,
 'Jan 19': 604812,
 'Feb 19': 604225,
 'Mar 19': 611026,
 'Apr 19': 614431,
 'May 19': 618306,
 'Jun 19': 615925,
 'Jul 19': 612743,
 'Aug 19': 610415,
 'Sep 19': 609588,
 'Oct 19': 607780,
 'Nov 19': 605625,
 'Dec 19': 601483,
 'Jan 20': 602718,
 'Feb 20': 604465,
 'Mar 20': 607748,
 'Apr 20': 609648,
 'May 20': 613454,
 'Jun 20': 609575,
 'Jul 20': 611713,
 'Aug 20': 638345,
 'Sep 20': 651060,
 'Oct 20': 658480,
 'Nov 20': 661667,
 'Dec 20': 665534,
 'Jan 21': 665201,
 'Feb 21': 666382,
 'Mar 21': 665473,
 'Apr 21': 672180,
 'May 21': 704383,
 'Jun 21': 697538,
 'Jul 21': 703877,
 'Aug 21': 701278,
 'Sep 21': 703183,
 'Oct 21': 745376,
 'Nov 21': 744866,
 'Dec 21': 736113,
 'Jan 22': 737828,
 'Feb 22': 743290,
 'Mar 22': 749833,
 'Apr 22': 757290,
 'May 22': 766331,
 'Jun 22': 788659,
 'Jul 22': 793321,
 'Aug 22': 780371,
 'Sep 22': 785725,
 'Oct 22': 794067,
 'Nov 22': 772435,
 'Dec 22': 777618,
 'Jan 23': 7

In [15]:
region_type=2
property_type=1

In [16]:
client.get_offer_insights(region_type=region_type,region_id=region_id,property_type=property_type)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): redfin.com:443
DEBUG:urllib3.connectionpool:https://redfin.com:443 "GET /stingray/api/region/2/8729/1/offer-insights HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.redfin.com:443
DEBUG:urllib3.connectionpool:https://www.redfin.com:443 "GET /stingray/api/region/2/8729/1/offer-insights HTTP/1.1" 200 1529


{'version': 494,
 'errorMessage': 'Success',
 'resultCode': 0,
 'payload': {'sections': [{'id': 353026,
    'agentInfoDisplay': {'agentName': 'Doug Ackerson',
     'isRedfinAgent': True,
     'isPartnerAgent': False,
     'isExternalAgent': False,
     'agentUrl': '/real-estate-agents/douglas-ackerson',
     'agentPhotoUrl': 'https://ssl.cdn-redfin.com/system_files/images/7129/150x150/gen120x120/1_6.jpg'},
    'story': 'Clients waived all contingencies and escalated to a number well above list price that they were comfortable with.  Unfortunately, there were other offerors that went higher.',
    'approxSalePrice': '$550K',
    'approxOfferPrice': '$550K',
    'won': False,
    'numBeds': 3,
    'numBaths': 3.5,
    'approxNumSqFt': '2,000',
    'offerToListRatio': '108%',
    'numDaysOnMarket': 5,
    'numCompetingOffers': '8',
    'downPaymentRange': '<10%',
    'dealId': 3905299,
    'homeInfo': {'searchStatus': 4, 'listingType': 1, 'propertyType': 13},
    'latLong': {'latitude': 3

### The entire webpage of data
https://www.redfin.com/stingray/api/home/details/belowTheFold?propertyId=9735814&accessLevel=1&listingId=170961922

### Time series of house price

This the list of the properties estimated value by month for the past 5 years. The last value is the curren MMM YY of the query

https://www.redfin.com/stingray/api/home/details/avmHistoricalData?propertyId=9735814&accessLevel=1&listingId=170961922

### If you are doing historical data you do not need the `listingId` parameter

https://www.redfin.com/stingray/api/home/details/avm?propertyId=9701815&accessLevel=1

belowTheFold returns house information, aboveTheFold returns images

### Required headers when trying to download the .csv

To download the csv you need the following header

* `headers = {'User-Agent': ''}`

When using the browser the full request header is 

```
headers = {Host': 'www.redfin.com',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/114.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1'}
```

Through trial and error I determined you only need `User-Agent` and the value can be empty.

## NOTE on building URL for table

It looks like you want `region_type=2` if you are searching a "`zip code`". I think `region_type=6` if you search "`city, state`"

From the search response

```
'version': 490,
 'errorMessage': 'Success',
 'resultCode': 0,
 'payload': {'sections': [{'rows': [{'id': '4_27862',
      'type': '4',
      'name': '64152',
      'subName': 'MO, USA',
      'url': '/zipcode/64152',
      'active': True,
      'claimedHome': False,
      'invalidMRS': False,
      'businessMarketIds': [53],
      'countryCode': 'US',
      'internalSearchVolume': 172}],
    'name': 'Places'}
```

You want `payload.sections[0].rows[0].id` and everything after the `_` in the case

`id': '4_27862'` gives you a `region_id` of `27862`. 

https://www.redfin.com/stingray/api/gis-csv?al=1&has_deal=false&has_dishwasher=false&has_laundry_facility=false&has_laundry_hookups=false&has_parking=false&has_pool=false&has_short_term_lease=false&include_pending_homes=false&isRentals=false&is_furnished=false&market=dc&num_homes=3500&ord=redfin-recommended-asc&page_number=1&region_id=27862&region_type=6&sold_within_days=1825&status=9&travel_with_traffic=false&travel_within_region=false&uipt=1,2,3,4,5,6,7,8&utilities_included=false&v=8


### url for zip
https://www.redfin.com/stingray/api/gis-csv?al=1&has_deal=false&has_dishwasher=false&has_laundry_facility=false&has_laundry_hookups=false&has_parking=false&has_pool=false&has_short_term_lease=false&include_pending_homes=false&isRentals=false&is_furnished=false&market=dc&num_homes=3500&ord=redfin-recommended-asc&page_number=1&region_id=8729&region_type=2&sold_within_days=1825&status=9&travel_with_traffic=false&travel_within_region=false&uipt=1,2,3,4,5,6,7,8&utilities_included=false&v=8

### url for city
https://www.redfin.com/stingray/api/gis-csv?al=1&has_deal=false&has_dishwasher=false&has_laundry_facility=false&has_laundry_hookups=false&has_parking=false&has_pool=false&has_short_term_lease=false&include_pending_homes=false&isRentals=false&is_furnished=false&market=dc&num_homes=350&ord=redfin-recommended-asc&page_number=1&region_id=21812&region_type=6&sold_within_days=1825&status=9&travel_with_traffic=false&travel_within_region=false&uipt=1,2,3,4,5,6,7,8&utilities_included=false&v=8

### How to find all the api calls the website is making

In [ ]:
page_reponse=requests.get('https://www.redfin.com/NY/Albany/363-Elk-St-12206/home/97942731', headers={'user-agent':''})

region type

id	type
1	neighborhood
2	ZIP code
5	County
6	City

Property type

uipt=1
id	type
1	House
2	Condo
3	Townhouse
4	Multi-family
5	Land
6	Other

Status

status=1
id	status
1	active
130	pending
131	active + pending

In [18]:
html_response = BeautifulSoup(page_reponse.text,"html.parser")

In [19]:
script_tags = html_response.find_all('script')

In [20]:
total_list=[]
for tag in script_tags:
    api_calls=[x.group() for x in re.finditer( r'stingray(.*?)\"postParams\"', str(tag))]
    if len(api_calls)>0:
        cleaned_apis=[]
        for i in api_calls:
            cleaned_apis.append(i.replace('\\u002F','/').replace('"',''))
        total_list+=cleaned_apis

In [21]:
total_list=list(set(total_list))

In [22]:
requests_list=[]
for i in total_list:
    try:
        call=json.loads('{"url":"'+i.split('{url:')[1].replace(',postParams','}}').replace('{','{"').replace(':','":"').replace(',','","').replace('}','"}').replace('"{"','{"').replace('"[','[').replace(']"',']').replace('}"}','}}'))
        requests_list.append(call)
    except:
        requests_list.append(i)

## Testing the functions

In [23]:
# response=client.get_similar_homes(property_id=property_id, listing_id=)

In [25]:
client.get_property_information(property_id=9736376)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): redfin.com:443
DEBUG:urllib3.connectionpool:https://redfin.com:443 "GET /stingray/api/home/details/belowTheFold?accessLevel=1&propertyId=9736376&listingId= HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.redfin.com:443
DEBUG:urllib3.connectionpool:https://www.redfin.com:443 "GET /stingray/api/home/details/belowTheFold?accessLevel=1&propertyId=9736376&listingId= HTTP/1.1" 200 None


{'version': 494,
 'errorMessage': 'Success',
 'resultCode': 0,
 'payload': {'amenitiesInfo': {'provider': '',
   'superGroups': [{'types': [21],
     'amenityGroups': [{'groupTitle': 'Basement Information',
       'referenceName': 'Basement Information',
       'amenityEntries': [{'amenityName': 'Basement Finish',
         'referenceName': 'BASEMENT_FINISH_CODE',
         'accessLevel': 1,
         'displayLevel': 1,
         'amenityValues': ['Full']},
        {'amenityName': 'Basement Type',
         'referenceName': 'BASEMENT_TYPE_CODE',
         'accessLevel': 1,
         'displayLevel': 1,
         'amenityValues': ['Full Daylight']}]},
      {'groupTitle': 'Bathroom Information',
       'referenceName': 'Bathroom Information',
       'amenityEntries': [{'amenityName': '# of Full Baths',
         'referenceName': 'FULL_BATHS',
         'accessLevel': 1,
         'displayLevel': 1,
         'amenityValues': ['2']}]},
      {'groupTitle': 'Heating & Cooling',
       'referenceName':